In [1]:
!pip install pypdf

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [3]:
!pip install sentence_transformers

In [4]:
!pip install llama_index

  Using cached llama_index_core-0.12.52.post1-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.52.post1-py3-none-any.whl (7.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.13.0
    Uninstalling llama-index-core-0.13.0:
      Successfully uninstalled llama-index-core-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-langchain 0.4.0 requires llama-index-core<0.14,>=0.13.0, but you have llama-index-core 0.12.52.post1 which is incompatible.
llama-index-llms-huggingface 0.6.0 requires llama-index-core<0.14,>=0.13.0, but you have llama-index-core 0.12.52.post1 which is incompatible.


In [5]:

!pip install llama-index-llms-huggingface



  Using cached llama_index_core-0.13.0-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.13.0-py3-none-any.whl (7.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.52.post1
    Uninstalling llama-index-core-0.12.52.post1:
      Successfully uninstalled llama-index-core-0.12.52.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-program-openai 0.3.2 requires llama-index-core<0.13,>=0.12.0, but you have llama-index-core 0.13.0 which is incompatible.
llama-index-llms-openai 0.4.7 requires llama-index-core<0.13,>=0.12.41, but you have llama-index-core 0.13.0 which is incompatible.
llama-index-multi-modal-llms-openai 0.5.3 requires llama-index-core<0.13,>=0.12.47, but you have llama-index-core 0.13.0 which is incompatible.
llama-index-embeddings-openai 0.3.1 requires llama-index-core<

In [6]:
# load the pdf
from llama_index.core import VectorStoreIndex,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
from llama_index.readers.file import PDFReader
reader=PDFReader()
documents=reader.load_data("attention.pdf")


In [7]:
documents


[Document(id_='2be22547-f0a4-44e1-81c5-b8771be43549', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction m

In [8]:
system_prompt= """
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
"""
#default format supported by llama2
Query_wrapper_Prompt=PromptTemplate("<|USER>{query_str}<|ASSISTANT>")

In [9]:
Query_wrapper_Prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER>{query_str}<|ASSISTANT>')

In [10]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [11]:
import torch
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=Query_wrapper_Prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model_kwargs={"torch_dtype":torch.float16,"load_in_8bit":True},
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#from llama_index.core.embeddings import LangchainEmbedding
from llama_index.core import ServiceContext
embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
#service_context=ServiceContext.from_defaults(embed_model=embed_model)

/tmp/ipython-input-3702576291.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [13]:
!pip install -U langchain-community

In [14]:
!pip install llama-index-embeddings-langchain

In [17]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#from llama_index.core.embeddings import LangchainEmbedding
from llama_index.core import Settings
embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
Settings.embed_model=embed_model
#settings=Settings(embed_model=embed_model)

In [19]:
index=VectorStoreIndex.from_documents(documents)
index

In [25]:
query_engine=index.as_query_engine()
response=query_engine.query("what is attention is all you need?")
print(response)

Attention is a mechanism used in machine learning models, particularly in natural language processing tasks. It allows the model to focus on specific parts of the input sequence when making predictions or generating outputs. The attention mechanism assigns weights to different elements of the input based on their relevance to the current step of processing. This helps the model to capture dependencies and relationships within the input data more effectively.
